# Extracting Tables from an Invoice Using pdfplumber

This code demonstrates how to extract tabular data, such as line items, prices, quantities, and totals, from an invoice using pyplumber. Extracting tables from invoices is useful in scenarios where structured data from financial documents is needed for analysis, automation, or integration with other systems.

## Use Case
- **Invoices**: Automatically extracting product descriptions, prices, quantities, and totals from invoices.
- **Financial Reports**: Extracting line-item details from financial statements.
- **Automation**: Automating data extraction from invoices for accounting, reporting, or auditing purposes.

In [13]:
import re
import pdfplumber
import pandas as pd

In [14]:
# Define the path to the PDF file
path = r"C:\Users\Quynh Pham\Desktop\Import pdf\Invoice2.pdf"

# Read in pdf file

In [17]:
# Open the PDF with pdfplumber
with pdfplumber.open(path) as pdf:
    # iterate over each page
    for page in pdf.pages:
        # extract text
        text = page.extract_text()
        print(text)

Denny Gunawan
221 Queen St
Melbourne VIC 3000
$39.60
123 Somewhere St, Melbourne VIC 3000
(03) 1234 5678
Invoice Number: #20130304
Organic Items Price/kg Quantity(kg) Subtotal
Apple $5.00 1 $5.00
Orange $1.99 2 $3.98
Watermelon $1.69 3 $5.07
Mango $9.56 2 $19.12
Peach $2.99 1 $2.99
Subtotal $36.00
GST (10%) $3.60
Total $39.60
* Lorem ipsum dolor sit amet, consectetur adipiscing elit. Aliquam sodales
dapibus fermentum. Nunc adipiscing, magna sed scelerisque cursus, erat
lectus dapibus urna, sed facilisis leo dui et ipsum.


## Extract tables

In [16]:
with pdfplumber.open(path) as pdf:
    # iterate over each page   
    for page in pdf.pages:
        # Extract tables from the page
        tables = page.extract_tables()

        # Display the extracted table rows
        for table in tables:
            for row in table:
                print(row)

['Organic Items', 'Price/kg', 'Quantity(kg)', 'Subtotal']
['Orange', '$1.99', '2', '$3.98']
['Mango', '$9.56', '2', '$19.12']
['', '', '']


**pdfplumber** provides a convenient way to extract tables using the `.extract_tables()` or `.extract_table()` methods. However, as seen in this case, the tables were not extracted accurately, with some values missing or misaligned. 

To address this, we can manually define a pattern for each line in the table and iterate through the entire PDF file to extract the data more accurately, ensuring that no values are missed. The code is as follows:

In [18]:
# Regular expression to capture the items, prices, quantity, and subtotal
pattern = r'([A-Za-z]+)\s+\$(\d+\.\d+)\s+(\d+)\s+\$(\d+\.\d+)'

# Find all matches of the pattern
matches = re.findall(pattern, text)

# Output the extracted items
print(matches)

[('Apple', '5.00', '1', '5.00'), ('Orange', '1.99', '2', '3.98'), ('Watermelon', '1.69', '3', '5.07'), ('Mango', '9.56', '2', '19.12'), ('Peach', '2.99', '1', '2.99')]


In [19]:
# Create a DataFrame from the extracted data
df = pd.DataFrame(matches, columns=['Item', 'Price/kg', 'Quantity(kg)', 'Subtotal'])

# Convert Price/kg, Quantity(kg), and Subtotal columns to numeric types
df['Price/kg'] = pd.to_numeric(df['Price/kg'])
df['Quantity(kg)'] = pd.to_numeric(df['Quantity(kg)'])
df['Subtotal'] = pd.to_numeric(df['Subtotal'])

# Print out the result
df

,Item,Price/kg,Quantity(kg),Subtotal
0,Apple,5.00,1,5.00
1,Orange,1.99,2,3.98
2,Watermelon,1.69,3,5.07
3,Mango,9.56,2,19.12
4,Peach,2.99,1,2.99
